In [ ]:
! pip install textalloc

In [ ]:
! pip install adjustText

In [ ]:
# ! scp -r newhpc:/home/sergeyv.venev-umw/projects/map_ranger_v034/mqc ./mqc_stats_v034
# ! ls -la ./mqc_stats_v034/multiqc_data
# ! head -100 ./mqc_stats_v034/multiqc_data/multiqc_general_stats.txt

In [ ]:
# ! ssh newhpc ls /home/sergeyv.venev-umw/projects/map_ranger_v034
# ! scp newhpc:/home/sergeyv.venev-umw/projects/map_ranger_v034/ranger.yml ./
# ! scp newhpc:/home/sergeyv.venev-umw/projects/map_ranger_v034/ranger_fastq.csv ./

In [ ]:
import yaml

with open('ranger.yml', 'r') as file:
    _ranger = yaml.safe_load(file)

# print(prime_service['prime_numbers'][0])
# print(prime_service['rest']['url'])

sample_catalog = []
for sample, lanes_dict in _ranger["input"]["raw_reads_paths"].items():
    for fastq_PE in lanes_dict.values():
        sample_catalog.append((sample, fastq_PE[0]))

sample_df = pd.DataFrame(sample_dict, columns=["sample", "fastq"])

rlen_df = pd.read_csv("ranger_fastq.csv", names=["rlen", "fastq"])[["fastq","rlen"]]

In [ ]:
# comprehensive panel of rlengths per sample ...
sample_df.merge(
    rlen_df,
    on="fastq"
).groupby("sample")["rlen"].apply(list)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# # download test data
# # this file is 145 Mb, and may take a few seconds to download
# import cooltools
# from cooltools import insulation

import bioframe
from matplotlib.colors import LogNorm
# from helper_func import saddleplot
from data_catalog import bws, bws_vlim, telo_dict, telo_reps_dict
# from helper_func import get_stack, show_stacks

import cooler
import cooltools.lib.plotting

from sklearn import decomposition
# unused but required import for doing 3d projections with matplotlib < 3.2
import mpl_toolkits.mplot3d  # noqa: F401

import textalloc as ta

In [ ]:
# define genomic view that will be used to call dots and pre-compute expected
# ...
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
# # remove "bad" chromosomes and near-empty arms ...
# excluded_arms = ["chr13_p", "chr14_p", "chr15_p", "chr21_p", "chr22_p", "chrM_p", "chrY_p", "chrY_q", "chrX_p", "chrX_q"]
# hg38_arms = hg38_arms_full[~hg38_arms_full["name"].isin(excluded_arms)].reset_index(drop=True)
# ...
# # can do 1 chromosome (or arm) as well ..
# included_arms = ["chr1_q", "chr2_p", "chr4_q", "chr6_q"]
included_arms = hg38_arms_full["name"].to_list()[:44] # all autosomal ones ...
hg38_arms = hg38_arms_full[hg38_arms_full["name"].isin(included_arms)].reset_index(drop=True)

# Load multiQC stats for visualizing ...

In [ ]:
df = pd.read_csv("./mqc_stats_v034/multiqc_data/multiqc_general_stats.txt",sep="\t")
df["Sample"] = df["Sample"].str.removesuffix(".hg38")
df = df.set_index("Sample")
df.columns = df.columns.str.replace("pairtools_mqc-generalstats-pairtools-", "")
# df

In [ ]:
# timecourse samples - control
_m_samples_pooled = [
    "ranGAP1-0-Ms-R1R2",
    "ranGAP1-0-Telo-R2",
    "ranGAP1-0-Cyto-R1",
    "ranGAP1-0-G1s-R1R2",
]
# timecourse samples rangap depletion
_p_samples_pooled = [
    "ranGAP1-2-Ms-R1R2",
    "ranGAP1-4-Cyto-R1",
    "ranGAP1-4-Telo-R2",
    "ranGAP1-7-G1s-R1R2",
]

# Nup 93 samples - control
_n93_m_samples_pooled = [
    "N93-0-G1s-R1R2",
]
# Nup 93 samples - depletion
_n93_p_samples_pooled = [
    "N93-7-G1s-R1R2",
]

# additional samples mp experiment
_mp_samples_pooled = [
    "ranGAP1-0-10hR-R1R3",
    "ranGAP1-12-10hR-R1R3",
    "ranGAP1-7-10hR-R1R3",
]

_pooled_idx = (
    _m_samples_pooled+
    _p_samples_pooled+
    _n93_m_samples_pooled+
    _n93_p_samples_pooled+
    _mp_samples_pooled
)
df.loc[_pooled_idx].to_csv("pooled-hic.csv")

In [ ]:
! readlink -f pooled-hic.csv
! readlink -f separate-hic.csv

In [ ]:
_idx = (
    _m_samples+
    _p_samples+
    _n93_m_samples+
    _n93_p_samples+
    _mp_samples
)
df.loc[_idx].to_csv("separate-hic.csv")

In [ ]:
# for s in df["Sample"]:
#     print(s)

In [ ]:
# timecourse samples - control
_m_samples = [
    "ranGAP1-0-Ms-R1",
    "ranGAP1-0-Ms-R2",
    "ranGAP1-0-Telo-R2",
    "ranGAP1-0-Cyto-R1",
    "ranGAP1-0-G1s-R1",
    "ranGAP1-0-G1s-R2",
]
# timecourse samples rangap depletion
_p_samples = [
    "ranGAP1-2-Ms-R1",
    "ranGAP1-2-Ms-R2",
    "ranGAP1-4-Cyto-R1",
    "ranGAP1-4-Telo-R2",
    "ranGAP1-7-G1s-R1",
    "ranGAP1-7-G1s-R2",
]

# Nup 93 samples - control
_n93_m_samples = [
    "N93-0-G1s-R1",
    "N93-0-G1s-R2",
]
# Nup 93 samples - depletion
_n93_p_samples = [
    "N93-7-G1s-R1",
    "N93-7-G1s-R2",
]


# additional samples mp experiment
_mp_samples = [
    "ranGAP1-0-10hR-R1",
    "ranGAP1-0-10hR-R3",
    "ranGAP1-12-10hR-R1",
    "ranGAP1-12-10hR-R3",
    "ranGAP1-7-10hR-R1",
    "ranGAP1-7-10hR-R3",
]

# Nup 10 hrs thing is excluded ...
# ...
# # Nup 93 samples - mp experiment - are we going to use these at all ?
# _n93_mp_samples = [
#     "N93-0-10hR-G1s-R3",
#     "N93-0-10hR-G1s-R4",
#     "N93-12-10hR-G1s-R3",
#     "N93-12-10hR-G1s-R4",
#     "N93-7-10hR-G1s-R3",
#     "N93-7-10hR-G1s-R4",
# ]


# entire 10 hours and mp thing for Nup93 are excluded ...
excluded_samples = [
    'N93p10_R1',
    'N93p10_R2',
    'N93mp10_R1',
    'N93mp10_R2',
    'N93m10_R1',
    'N93m10_R2',
    'N93m10',
    'N93p10',
    'N93mp10',
]


# map to sapmples used in the paper ...
smap = {
    "ranGAP1-0-Ms-R1" : "cMito",
    "ranGAP1-0-Ms-R2" : "cMito",
    "ranGAP1-0-Telo-R2" : "cTelo",
    "ranGAP1-0-Cyto-R1" : "cCyto",
    "ranGAP1-0-G1s-R1" : "c5h",
    "ranGAP1-0-G1s-R2" : "c5h",
    "ranGAP1-2-Ms-R1" : "$\\Delta$Mito",
    "ranGAP1-2-Ms-R2" : "$\\Delta$Mito",
    "ranGAP1-4-Cyto-R1" : "$\\Delta$Cyto",
    "ranGAP1-4-Telo-R2" : "$\\Delta$Telo",
    "ranGAP1-7-G1s-R1" : "$\\Delta$5h",
    "ranGAP1-7-G1s-R2" : "$\\Delta$5h",
    "N93-0-G1s-R1" : "c5h_Nup93",
    "N93-0-G1s-R2" : "c5h_Nup93",
    "N93-7-G1s-R1" : "$\\Delta$5h_Nup93",
    "N93-7-G1s-R2" : "$\\Delta$5h_Nup93",
    "ranGAP1-0-10hR-R1" : "c10h",
    "ranGAP1-0-10hR-R3" : "c10h",
    "ranGAP1-12-10hR-R1" : "c5h+$\\Delta$10h",
    "ranGAP1-12-10hR-R3" : "c5h+$\\Delta$10h",
    "ranGAP1-7-10hR-R1" : "$\\Delta$10h",
    "ranGAP1-7-10hR-R3" : "$\\Delta$10h",
}


# map to sapmples used in the paper ...
smap2 = {
    "mMito": "cMito",
    "mTelo": "cTelo",
    "mCyto": "cCyto",
    "m5hR1R2": "c5h",
    "pMito": "$\\Delta$Mito",
    "pTelo": "$\\Delta$Telo",
    "pCyto": "$\\Delta$Cyto",
    "p5hR1R2": "$\\Delta$5h",
    "m10hR1R2": "c10h",
    "p10hR1R2": "$\\Delta$10h",
    "mp10hR1R2": "c5h+$\\Delta$10h",
    "N93m5": "c5h_Nup93",
    "N93p5": "$\\Delta$5h_Nup93",
}


smap2_reps = {
    'mMito_R1': 'cMito-R1',
    'mMito_R2': 'cMito-R2',
    'mTelo': 'cTelo-R1',
    'mCyto': 'cCyto-R1',
    'm5h_R1': 'c5h-R1',
    'm5h_R2': 'c5h-R2',
    'm10h_R1': 'c10h-R1',
    'm10h_R2': 'c10h-R2',
    'pMito_R1': '$\\Delta$Mito-R1',
    'pMito_R2': '$\\Delta$Mito-R2',
    'pTelo': '$\\Delta$T-R1',
    'pCyto': '$\\Delta$C-R1',
    'p5h_R1': '$\\Delta$5h-R1',
    'p5h_R2': '$\\Delta$5h-R2',
    'p10h_R1': '$\\Delta$10h-R1',
    'p10h_R2': '$\\Delta$10h-R2',
    'mp10h_R1': 'c5h+$\\Delta$10h-R1',
    'mp10h_R2': 'c5h+$\\Delta$10h-R2',
    'N93m5_R1': 'c5h_Nup93-R1',
    'N93m5_R2': 'c5h_Nup93-R2',
    'N93p5_R1': '$\\Delta$5h_Nup93-R1',
    'N93p5_R2': '$\\Delta$5h_Nup93-R2',
}

# combine smap ...
smap_combo = {
    "mMito": "cMito",
    "mTelo": "cTelo",
    "mCyto": "cCyto",
    "m5hR1R2": "c5h",
    "pMito": "$\\Delta$Mito",
    "pTelo": "$\\Delta$Telo",
    "pCyto": "$\\Delta$Cyto",
    "p5hR1R2": "$\\Delta$5h",
    "m10hR1R2": "c10h",
    "p10hR1R2": "$\\Delta$10h",
    "mp10hR1R2": "c5h+$\\Delta$10h",
    "N93m5": "c5h_Nup93",
    "N93p5": "$\\Delta$5h_Nup93",
    'mMito_R1': 'cMito-R1',
    'mMito_R2': 'cMito-R2',
    'm5h_R1': 'c5h-R1',
    'm5h_R2': 'c5h-R2',
    'm10h_R1': 'c10h-R1',
    'm10h_R2': 'c10h-R2',
    'pMito_R1': '$\\Delta$Mito-R1',
    'pMito_R2': '$\\Delta$Mito-R2',
    'p5h_R1': '$\\Delta$5h-R1',
    'p5h_R2': '$\\Delta$5h-R2',
    'p10h_R1': '$\\Delta$10h-R1',
    'p10h_R2': '$\\Delta$10h-R2',
    'mp10h_R1': 'c5h+$\\Delta$10h-R1',
    'mp10h_R2': 'c5h+$\\Delta$10h-R2',
    'N93m5_R1': 'c5h_Nup93-R1',
    'N93m5_R2': 'c5h_Nup93-R2',
    'N93p5_R1': '$\\Delta$5h_Nup93-R1',
    'N93p5_R2': '$\\Delta$5h_Nup93-R2',
}


repeated_samples = ["mTelo", "mCyto", "pTelo", "pCyto", ]

In [ ]:
_height = 0.8

# the groups we're actually going to show
_sample_groups = [
    _m_samples,
    _p_samples,
    _n93_m_samples,
    _n93_p_samples,
    _mp_samples,
]

fig, axs = plt.subplots(
    nrows=len(_sample_groups),
    ncols=4,
    sharex="col",
    sharey="row",
    height_ratios=[ len(g)+2*_height for g in _sample_groups ],
    figsize=(8,7)
)

barh_kwargs = dict(
    align="center",
    height=_height,
)

# total reads go first
for i, _samples in enumerate(_sample_groups):
    ax = axs[i,0]
    ax.barh(
        np.arange(len(_samples)),
        df.loc[_samples, "total"]/1_000_000_000,
        **barh_kwargs,
    )
    ax.set_yticks(np.arange(len(_samples)), labels=_samples)
    ax.set_ylim(0-_height,len(_samples)-1+_height)

ax.set_xticks([0,1,2,3])
ax.set_xlabel("sequenced\nPE reads, billions", fontsize=10)



# percetn usable reads
for i, _samples in enumerate(_sample_groups):
    ax = axs[i,1]
    ax.barh(
        np.arange(len(_samples)),
        100.*(df.loc[_samples, "total_nodups"]/df.loc[_samples, "total"]),
        **barh_kwargs,
    )
    ax.set_yticks(np.arange(len(_samples)), labels=_samples)
    ax.invert_yaxis()
    ax.set_ylim(0-_height,len(_samples)-1+_height)
ax.set_xlabel("unique interactions\naka usable pairs, %", fontsize=10)


# cis trans ...
for i, _samples in enumerate(_sample_groups):
    ax = axs[i,2]
    ax.barh(
        np.arange(len(_samples)),
        df.loc[_samples, "frac_cis"],
        left=[0]*len(_samples),
        color="tab:blue",
        **barh_kwargs,
    )
    ax.barh(
        np.arange(len(_samples)),
        100. - df.loc[_samples, "frac_cis"],
        left=df.loc[_samples, "frac_cis"],
        color="tab:brown",
        **barh_kwargs,
    )
    ax.set_ylim(0-_height,len(_samples)-1+_height)
    ax.invert_yaxis()
ax.set_xlabel("cis + trans, %", fontsize=10);



# total reads go first
for i, _samples in enumerate(_sample_groups):
    ax = axs[i,3]
    for t,s in enumerate(_samples):
        ax.text(0.033,t+0.25,smap[s])
    # ax.set_yticks(np.arange(len(_samples)), labels=_samples)
    ax.set_ylim(0-_height,len(_samples)-1+_height)
    ax.set_xlim(0,1)
    ax.invert_yaxis()
    ax.set_xticks([])
ax.set_xlabel("pooled\nsample names", fontsize=10,labelpad=20)

fig.savefig("SuppFig1_samples.svg",dpi=300)

In [ ]:
# [0]*len(_samples)

# ! ls ins*
# ! ls ev*

# ev_bedraph
# ev_r1r2_bedraph
# ins_bedgraph
# ins_r1r2_bedgraph

# telo_dict,
# telo_reps_dict
# telo_reps_cis_eigs_gene[binsize][sample_m]

## Pre-load pre-calculated EV1s and coolers - for whatever resolution ...

In [ ]:
binsize25 = 25_000

In [ ]:
# ...
telo_cis_evs = {}
for k, _fname in telo_dict.items():
    # derive output name
    _fname = f"ev_bedraph/{k}.{binsize25//1_000}kb.bed"
    print(f"reading {_fname} ...")
    telo_cis_evs[k] = bioframe.read_table(_fname, schema="bedGraph", header=0)
# #
# # cooler files that we'll work on :
# telo_clrs = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize25}") for _k, _path in telo_dict.items() }

In [ ]:
# ...
telo_reps_cis_evs = {}
for k, _fname in telo_reps_dict.items():
    # derive output name
    _fname = f"ev_r1r2_bedraph/{k}.{binsize25//1_000}kb.bed"
    print(f"reading {_fname} ...")
    telo_reps_cis_evs[k] = bioframe.read_table(_fname, schema="bedGraph", header=0)
# #
# # cooler files that we'll work on :
# telo_clrs = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize25}") for _k, _path in telo_dict.items() }

# Read Insulation tracks - only relevant 4 columns - aka bedGraph ...

In [ ]:
binsize10 = 10_000
diamond_size = 10*binsize10
ins_value_col = f"log2_insulation_score_{diamond_size}"
cols = ["chrom", "start", "end", value_colname]

In [ ]:
telo_ins = {}
for k, _fname in telo_dict.items():
    # derive output name
    in_fname = f"ins_bedgraph/{k}.{binsize10//1_000}kb.bed"
    telo_ins[k] = pd.read_csv(in_fname, sep="\t")[cols]
    print(f"read {in_fname} ...")

In [ ]:
telo_reps_ins = {}
for k, _fname in telo_reps_dict.items():
    # derive output name
    in_fname = f"ins_r1r2_bedgraph/{k}.{binsize10//1_000}kb.bed"
    telo_reps_ins[k] = pd.read_csv(in_fname, sep="\t")[cols]
    print(f"read {in_fname} ...")

# Cluster/PCA our samples to show which ones are more similar (insulations first)...

In [ ]:
_pca_samples = [
    "m5h_R1",
    "m5h_R2",
    "p5h_R1",
    "p5h_R2",
    "N93m5_R1",
    "N93m5_R2",
    "N93p5_R1",
    "N93p5_R2",
]
_rest_samples = [_sample for _sample in _all_samples if (_sample not in _pca_samples)and(_sample not in excluded_samples)]


In [ ]:

_value_col = "value"
_common_evs_df = pd.concat(
    [ _df[_value_col].rename(_sample) for _sample, _df in telo_reps_cis_evs.items() ],
    axis=1,
)
# ...
# combine pooled and separated ...
_good_bins_common = ~_common_evs_df.isna().any(axis=1).to_numpy()
# #
# # ...
# #
_all_samples = [_sample for _sample in telo_reps_cis_evs]

In [ ]:
X = _common_evs_df[_pca_samples].to_numpy()[_good_bins_common].T
X_rest = _common_evs_df[_rest_samples].to_numpy()[_good_bins_common].T

# do PCA on X ...
pcaX = decomposition.PCA(n_components=3)
print(f"running PCA on {_pca_samples=} ...")
pcaX.fit(X)

# project X and X_rest onto PCs ...
X_trans = pcaX.transform(X)
X_rest_trans = pcaX.transform(X_rest)

print(pcaX.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

_x, _y, _ = X_trans.T
_x_rest, _y_rest, _ = X_rest_trans.T

text_list = []
for txt in _pca_samples:
    text_list.append(smap_combo[txt])
for txt in _rest_samples:
    text_list.append(smap_combo[txt])

ax.scatter(_x, _y, s=100, color="tab:red", alpha=0.9, edgecolors="dimgray")
ax.scatter(_x_rest, _y_rest, s=100, color="tab:blue", alpha=0.9, edgecolors="dimgray")

# texts = [ax.text(_x[i], _y[i], smap_combo[txt], ha='center', va='center') for i, txt in enumerate(_pca_samples)]
# texts += [ax.text(_x_rest[i], _y_rest[i], smap_combo[txt], ha='center', va='center') for i, txt in enumerate(_rest_samples)]
texts = [ax.text(_x[i], _y[i], txt, ha='center', va='center') for i, txt in enumerate(_pca_samples)]
texts += [ax.text(_x_rest[i], _y_rest[i], txt, ha='center', va='center') for i, txt in enumerate(_rest_samples)]

# adjust_text(texts);
adjust_text(
    texts,
    expand=(1.5, 2.5), # expand text bounding boxes by 1.2 fold in x direction and 2 fold in y direction
    arrowprops=dict(arrowstyle='->', color='gray') # ensure the labeling is clear by adding arrows
);

# ...
# plt.scatter(_x, _y, s=50, color="blue")
# for i, txt in enumerate(_rest_samples):
#     ax.annotate(smap_combo[txt], (_x[i], _y[i]), fontsize=9)

_1, _2, _3 = pca.explained_variance_ratio_[:3]
ax.set_xlabel(f"PC1: {_1:.2f}", fontsize=13)
ax.set_ylabel(f"PC2: {_2:.2f}", fontsize=13)


fig.savefig("SuppFig1_evclust.svg", dpi=300)

# Cluster/PCA our samples to show which ones are more similar (Insulation second)...

In [ ]:

_value_col = ins_value_col
_common_ins_df = pd.concat(
    [ _df[_value_col].rename(_sample) for _sample, _df in telo_reps_ins.items() ],
    axis=1,
)
# ...
# combine pooled and separated ...
_good_bins_common = ~_common_ins_df.isna().any(axis=1).to_numpy()
# #
# # ...
# #
_all_samples = [_sample for _sample in telo_reps_ins]

In [ ]:
X = _common_ins_df[_pca_samples].to_numpy()[_good_bins_common].T
X_rest = _common_ins_df[_rest_samples].to_numpy()[_good_bins_common].T

# do PCA on X ...
pcaX = decomposition.PCA(n_components=3)
print(f"running PCA on {_pca_samples=} ...")
pcaX.fit(X)

# project X and X_rest onto PCs ...
X_trans = pcaX.transform(X)
X_rest_trans = pcaX.transform(X_rest)

print(pcaX.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

_x, _y, _ = X_trans.T
_x_rest, _y_rest, _ = X_rest_trans.T

text_list = []
for txt in _pca_samples:
    text_list.append(smap_combo[txt])
for txt in _rest_samples:
    text_list.append(smap_combo[txt])

ax.scatter(_x, _y, s=100, color="tab:red", alpha=0.9, edgecolors="dimgray")
ax.scatter(_x_rest, _y_rest, s=100, color="tab:blue", alpha=0.9, edgecolors="dimgray")

# texts = [ax.text(_x[i], _y[i], smap_combo[txt], ha='center', va='center') for i, txt in enumerate(_pca_samples)]
# texts += [ax.text(_x_rest[i], _y_rest[i], smap_combo[txt], ha='center', va='center') for i, txt in enumerate(_rest_samples)]
texts = [ax.text(_x[i], _y[i], txt, ha='center', va='center') for i, txt in enumerate(_pca_samples)]
texts += [ax.text(_x_rest[i], _y_rest[i], txt, ha='center', va='center') for i, txt in enumerate(_rest_samples)]

# adjust_text(texts);
adjust_text(
    texts,
    expand=(1.5, 2.5), # expand text bounding boxes by 1.2 fold in x direction and 2 fold in y direction
    arrowprops=dict(arrowstyle='->', color='gray') # ensure the labeling is clear by adding arrows
);

# ...
# plt.scatter(_x, _y, s=50, color="blue")
# for i, txt in enumerate(_rest_samples):
#     ax.annotate(smap_combo[txt], (_x[i], _y[i]), fontsize=9)

_1, _2, _3 = pca.explained_variance_ratio_[:3]
ax.set_xlabel(f"PC1: {_1:.2f}", fontsize=13)
ax.set_ylabel(f"PC2: {_2:.2f}", fontsize=13)

fig.savefig("SuppFig1_insclust.svg", dpi=300)

# OLD STUFF - don't worry about it ...

# Cluster/PCA our samples to show which ones are more similar (EVs first)...

In [ ]:
_value_col = "value"
_common_evs_df = pd.concat(
    (
        [_df[_value_col].rename(_sample) for _sample, _df in telo_cis_evs.items()] +
        [_df[_value_col].rename(_sample) for _sample, _df in telo_reps_cis_evs.items() if _sample not in repeated_samples]
    ),
    axis=1,
)
# ...
# combine pooled and separated ...
_good_bins_common = ~_common_evs_df.isna().any(axis=1).to_numpy()
#
# ...
#
_all_samples = (
    [_sample for _sample in telo_cis_evs] +
    [_sample for _sample in telo_reps_cis_evs if _sample not in repeated_samples]
)

# Cluster/PCA our samples to show which ones are more similar (EVs first)...

In [ ]:
_value_col = "value"
_common_evs_df = pd.concat(
    (
        [_df[_value_col].rename(_sample) for _sample, _df in telo_cis_evs.items()] +
        [_df[_value_col].rename(_sample) for _sample, _df in telo_reps_cis_evs.items() if _sample not in repeated_samples]
    ),
    axis=1,
)
# ...
# combine pooled and separated ...
_good_bins_common = ~_common_evs_df.isna().any(axis=1).to_numpy()
#
# ...
#
_all_samples = (
    [_sample for _sample in telo_cis_evs] +
    [_sample for _sample in telo_reps_cis_evs if _sample not in repeated_samples]
)

In [ ]:
# _common_evs_df.columns
# len([_df[_value_col].rename(_sample) for _sample, _df in telo_cis_evs.items()])
# len([_df[_value_col].rename(_sample) for _sample, _df in telo_reps_cis_evs.items() if _sample not in repeated_samples])

In [ ]:
_pca_samples = [
    # 'mTelo',
    # 'mCyto',
    'm5hR1R2',
    'm10hR1R2',
    # 'pTelo',
    # 'pCyto',
    'p5hR1R2',
    'p10hR1R2',
    # 'mp10hR1R2',
    'N93m5',
    'N93p5',
]

_rest_samples = [_sample for _sample in _all_samples if (_sample not in _pca_samples)and(_sample not in excluded_samples)]

X = _common_evs_df[_pca_samples].to_numpy()[_good_bins_common].T
X_rest = _common_evs_df[_rest_samples].to_numpy()[_good_bins_common].T

# do PCA on X ...
pcaX = decomposition.PCA(n_components=3)
print(f"running PCA on {_pca_samples=} ...")
pcaX.fit(X)

# project X and X_rest onto PCs ...
X_trans = pcaX.transform(X)
X_rest_trans = pcaX.transform(X_rest)

print(pcaX.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

_x, _y, _ = X_trans.T
_x_rest, _y_rest, _ = X_rest_trans.T

text_list = []
for txt in _pca_samples:
    text_list.append(smap_combo[txt])
for txt in _rest_samples:
    text_list.append(smap_combo[txt])

_xx = np.r_[_x, _x_rest]
_yy = np.r_[_y, _y_rest]

ax.scatter(_x, _y, s=60, color="tab:red", alpha=0.8)
ax.scatter(_x_rest, _y_rest, s=60, color="tab:blue", alpha=0.8)

ta.allocate(
    ax,
    _xx,
    _yy,
    text_list,
    # x_scatter=_xx,
    # y_scatter=_yy,
    linecolor="black",
    textsize=9
)
# ...
# plt.scatter(_x, _y, s=50, color="blue")
# for i, txt in enumerate(_rest_samples):
#     ax.annotate(smap_combo[txt], (_x[i], _y[i]), fontsize=9)

_1, _2, _3 = pca.explained_variance_ratio_[:3]
ax.set_xlabel(f"PC1: {_1:.2f}")
ax.set_ylabel(f"PC2: {_2:.2f}")

In [ ]:
# ta.allocate?

In [ ]:
_value_col = "value"


_common_mask = pd.concat(
    [_df[_value_col].rename(_sample) for _sample, _df in telo_reps_cis_evs.items()],
    axis=1,
).isna().any(axis=1).to_numpy()





_pca_samples = [
    # 'mMito_R1',
    # 'mMito_R2',
    # 'mTelo',
    # 'mCyto',
    'm5h_R1',
    'm5h_R2',
    'm10h_R1',
    'm10h_R2',
    # 'pMito_R1',
    # 'pMito_R2',
    # 'pTelo',
    # 'pCyto',
    'p5h_R1',
    'p5h_R2',
    'p10h_R1',
    'p10h_R2',
    # 'mp10h_R1',
    # 'mp10h_R2',
    'N93m5_R1',
    'N93m5_R2',
    'N93m10_R1',
    'N93m10_R2',
    'N93p5_R1',
    'N93p5_R2',
    'N93p10_R1',
    'N93p10_R2',
    # 'N93mp10_R1',
    # 'N93mp10_R2',
]

# _pca_samples = [
#     # 'mTelo',
#     # 'mCyto',
#     'm5hR1R2',
#     # 'm10hR1R2',
#     # 'pTelo',
#     # 'pCyto',
#     'p5hR1R2',
#     # 'p10hR1R2',
#     # 'mp10hR1R2',
#     'N93m5',
#     # 'N93m10',
#     # 'N93p5',
#     'N93p10',
#     # 'N93mp10',
# ]

_rest_samples = [_sample for _sample in telo_reps_cis_evs if (_sample not in _pca_samples)]


X = pd.concat(
    [telo_reps_cis_evs[_sample][_value_col].rename(_sample) for _sample in _pca_samples],
    axis=1,
).to_numpy()[~_common_mask].T



X_rest = pd.concat(
    [telo_reps_cis_evs[_sample][_value_col].rename(_sample) for _sample in _rest_samples],
    axis=1,
).to_numpy()[~_common_mask].T



pca = decomposition.PCA(n_components=3)
print("running PCA ...")
pca.fit(X)
X_trans = pca.transform(X)
X_rest_trans = pca.transform(X_rest)

print(pca.explained_variance_ratio_)

In [ ]:
from adjustText import adjust_text

In [ ]:

_x, _y, _z = X_trans[:,0], X_trans[:,1], X_trans[:,2]
plt.scatter(_x, _y, s=50, color="red")
ax = plt.gca()
# for i, txt in enumerate(_pca_samples):
#     ax.annotate(txt, (_x[i]+1, _y[i]+1), fontsize=9)

texts = [ax.text(_x[i], _y[i], txt, ha='center', va='center') for i, txt in enumerate(_pca_samples)]


_x, _y, _z = X_rest_trans[:,0], X_rest_trans[:,1], X_rest_trans[:,2]
plt.scatter(_x, _y, s=50, color="blue")
# for i, txt in enumerate(_rest_samples):
#     ax.annotate(txt, (_x[i]+1, _y[i]+1), fontsize=9)
texts += [ax.text(_x[i], _y[i], txt, ha='center', va='center') for i, txt in enumerate(_rest_samples)]


# adjust_text(texts);
adjust_text(
    texts,
    expand=(1.2, 2), # expand text bounding boxes by 1.2 fold in x direction and 2 fold in y direction
    arrowprops=dict(arrowstyle='->', color='grey') # ensure the labeling is clear by adding arrows
);

_1, _2, _3 = pca.explained_variance_ratio_[:3]
ax.set_xlabel(f"PC1: {_1:.2f}")
ax.set_ylabel(f"PC2: {_2:.2f}")


In [ ]:

sub_samples_m = [
    "mMito",
    "mTelo",
    "mCyto",
    "m5hR1R2",
    # "m10hR1R2",
]
sub_samples_p = [
    "pMito",
    "pTelo",
    "pCyto",
    "p5hR1R2",
    # "p10hR1R2",
]


ddd = {}
for sample in sub_samples_m+sub_samples_p:
    ins = telo_ins[sample]
    # ddd[sample] = bioframe.select(ins, the_region)[value_col]
    ddd[sample] = ins[value_col]

sns.heatmap(pd.DataFrame(ddd).corr(method='pearson'), annot=True, cmap="Reds",vmin=0.5,vmax=0.99)


In [ ]:

ord_samples = [
    "mMito",
    "pMito",
    "mTelo",
    "pTelo",
    "mCyto",
    "pCyto",
    "m5hR1R2",
    "p5hR1R2",
    # "m10hR1R2",
    # "p10hR1R2",
]

ddd = {}
for sample in ord_samples:
    ins = telo_ins[sample]
    # ddd[sample] = bioframe.select(ins, the_region)[value_col]
    ddd[sample] = ins[value_col]

sns.heatmap(pd.DataFrame(ddd).corr(method='pearson'), annot=True, cmap="Reds",vmin=0.5,vmax=0.95)


In [ ]:
sub_samples_m = [
    "mMito_R1",
    "mMito_R2",
    "mTelo",
    "mCyto",
    "m5h_R1",
    "m5h_R2",
    "m10h_R1",
    "m10h_R2",
]
# sub_samples_p = [
#     "pMito_R1",
#     "pMito_R2",
#     "pTelo",
#     "pCyto",
#     "p5h_R1",
#     "p5h_R2",
#     "p10h_R1",
#     "p10h_R2",
# ]



ddd = {}
for sample in sub_samples_m:
    ins = telo_reps_ins[sample]
    # ddd[sample] = bioframe.select(ins, the_region)[value_col]
    ddd[sample] = ins[value_col]

sns.heatmap(pd.DataFrame(ddd).corr(method='pearson'), annot=True, cmap="Reds",vmin=0.3,vmax=1)


In [ ]:
# sub_samples_m = [
#     "mMito_R1",
#     "mMito_R2",
#     "mTelo",
#     "mCyto",
#     "m5h_R1",
#     "m5h_R2",
#     "m10h_R1",
#     "m10h_R2",
# ]
sub_samples_p = [
    "pMito_R1",
    "pMito_R2",
    "pTelo",
    "pCyto",
    "p5h_R1",
    "p5h_R2",
    "p10h_R1",
    "p10h_R2",
]



ddd = {}
for sample in sub_samples_p:
    ins = telo_reps_ins[sample]
    # ddd[sample] = bioframe.select(ins, the_region)[value_col]
    ddd[sample] = ins[value_col]

sns.heatmap(pd.DataFrame(ddd).corr(method='pearson'), annot=True, cmap="Reds",vmin=0.3,vmax=1)


In [ ]:
sub_samples_m = [
    "mMito_R1",
    "mMito_R2",
    "mTelo",
    "mCyto",
    "m5h_R1",
    "m5h_R2",
    "m10h_R1",
    "m10h_R2",
]
sub_samples_p = [
    "pMito_R1",
    "pMito_R2",
    "pTelo",
    "pCyto",
    "p5h_R1",
    "p5h_R2",
    "p10h_R1",
    "p10h_R2",
]



ddd = {}
for sample in sub_samples_m+sub_samples_p:
    ins = telo_reps_ins[sample]
    # ddd[sample] = bioframe.select(ins, the_region)[value_col]
    ddd[sample] = ins[value_col]

sns.heatmap(pd.DataFrame(ddd).corr(method='pearson'), annot=True, cmap="Reds",vmin=0.3,vmax=1)
